# Herbert - Polish RoBERT'a model 


## Prepare the data.

* Wikipedia, Link: 
* Oscar
* Books, Polish "Wolne lektury"



Usefull resources


* https://github.com/pytorch/fairseq/blob/master/examples/roberta/README.pretraining.md
* https://github.com/musixmatchresearch/umberto/issues/2


In [1]:
import csv
import sys
import datetime as dt
import os
from pathlib import Path
import re

#from tqdm.notebook import tqdm
from tqdm import tqdm


import mmap

print("import")

import


## Create vocabulary


### Prepare data for vocab

Create files for training vocabulary. One sentence per line, use polish sentence tokenizer. 
Sentence piece model is capable of handling around 12_000_000 sentences, so more sentences is not necessary.





### Train the BPE sentencepiece model

We used the [sentencepiece](https://github.com/google/sentencepiece)


This can be done in two ways:
- as a python module
- as a command line 


If you want to use it as a commandline you should [build the C++ version from source](https://github.com/google/sentencepiece#c-from-source)


Lower txt file 

```
tr [:upper:] [:lower:] < corpus_books_wiki_12M_lines.txt > corpus_books_wiki_12M_lines_lower.txt
```



Train wikipedia vocab using commandline

```
spm_train \
    --input=./data/corpus_raw/corpus_books_wiki_12M_lines.txt \
    --max_sentence_length=4192\
    --model_prefix=./data/books_wikipedia_v32k_sen10M.spm.bpe \
    --vocab_size=32000 \
    --model_type=bpe \
    --shuffle_input_sentence=true \
    --input_sentence_size=10000000 \
    --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3
```



```
spm_train \
    --input=corpus_books_wiki_12M_lines_lower.txt \
    --max_sentence_length=4192\
    --model_prefix=books_wikipedia_lower_v32k_sen10M.spm.bpe \
    --vocab_size=32000 \
    --model_type=bpe \
    --shuffle_input_sentence=true \
    --input_sentence_size=10000000 \
    --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3
```


In [9]:
import sentencepiece as spm
vocab_size = 32000
model_type = "bpe"  
iss = 10_000_000

vocab_type = 'upper' 
#vocab_type = 'lower' 


tok_model = f"wikipedia_{vocab_type}_voc_{vocab_size}_sen{iss}"

data_file = './data/corpus_wikipedia_2020-02-01_lines.txt'

tok_model = os.path.abspath(f"./data/{tok_model}")

piece_options = ' --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3 --shuffle_input_sentence=true'


cmd = f"--input={data_file} --model_prefix={tok_model} --model_type={model_type} --num_threads=4 --vocab_size={vocab_size}  --input_sentence_size={iss}"+ piece_options
print(cmd)

start = dt.datetime.now()
print(start)
spm.SentencePieceTrainer.train(cmd)
end = dt.datetime.now()

print(f"Vocab of {vocab_size} tokens from {data_file} create takes {end-start}")





--input=./data/corpus_wikipedia_2020-02-01_lines.txt --model_prefix=/home/ksopyla/dev/ermlab/herbert/data/wikipedia_upper_voc_32000_sen10000000 --model_type=bpe --num_threads=4 --vocab_size=32000  --input_sentence_size=10000000 --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3 --shuffle_input_sentence=true
2020-02-27 13:11:27.769435
Vocab of 32000 tokens from ./data/corpus_wikipedia_2020-02-01_lines.txt create takes 0:13:14.934491


In [10]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load(f"{tok_model}.model")

# returns vocab size
print(sp.get_piece_size())
text = """Będąc młodym programistą (hoho), czytałem "Dziady" w 1983r."""

# encode: text => id
if vocab_type=="lower":
    text = text.lower()

print(sp.encode_as_pieces(text))

32000
['▁Będąc', '▁młodym', '▁programi', 'stą', '▁(', 'ho', 'ho', '),', '▁czy', 't', 'ałem', '▁"', 'D', 'zia', 'dy', '"', '▁w', '▁1983', 'r', '.']


### Fairseq vocab 

Change the bpe sentecepiece vocab separator form \t to space. It is format needed by fairseq-preprocess 
Open vocab file and replce \t 

In [12]:
 
vocab_file = './data/wiki_books_model/vocab/books_wikipedia_v32k_sen10M.spm.bpe.vocab'


p = Path(vocab_file)

output_path = f"{p.with_suffix('')}_fair.vocab"
with open(output_path, 'w+') as output_file:
    with open(vocab_file) as f:
        
        text = f.read().replace('\t',' ')
        output_file.write(text)


### Encode data with sentence piece model


Encode wiki 
```
# Encode Data with SentencePiece Tokenizer
cd data/wiki_model

for SPLIT in train valid test; do \
    spm_encode \
    --model=./vocab/wikipedia_upper_voc_32000_sen10000000.model \
    --extra_options=bos:eos \
    --output_format=piece \
    < corpus_wikipedia_2020-02-01_${SPLIT}.txt \
    > corpus_wikipedia_2020-02-01_${SPLIT}.txt.bpe
done
```


**Encode wiki books oscar**

```
# Encode Data with SentencePiece Tokenizer
cd data/wiki_books_oscar_model

for SPLIT in test train ; do \
    spm_encode \
    --model=./vocab/books_wikipedia_v32k_sen10M.spm.bpe.model \
    --extra_options=bos:eos \
    --output_format=piece \
    < corpus_wiki_books_oscar_${SPLIT}.txt \
    > corpus_wiki_books_oscar_${SPLIT}.txt.bpe
done
```

### Preprocess data with fairseq

In [1]:

!pipenv install fairseq


Installing fairseq...
Adding fairseq to Pipfile's [packages]...
✔ Installation Succeeded 
Installing dependencies from Pipfile.lock (d640ed)...



## Fairseq  preprocess bpe encoded and splited data


Wikipedia 2020-020-01 small(test) model

```
fairseq-preprocess \
    --only-source \
    --srcdict ./vocab/wikipedia_upper_voc_32000_sen10000000_fair.vocab \
    --trainpref corpus_wikipedia_2020-02-01_train.txt.bpe \
    --validpref corpus_wikipedia_2020-02-01_valid.txt.bpe \
    --testpref corpus_wikipedia_2020-02-01_test.txt.bpe \
    --destdir ./ \
    --workers 3
```


Wiki books oscar data

Vocab32k process data

```
fairseq-preprocess \
    --only-source \
    --srcdict ./vocab/books_wikipedia_v32k_sen10M.spm.bpe_fair.vocab \
    --trainpref corpus_wiki_books_oscar_train_vocab32k.txt.bpe \
    --validpref corpus_wiki_books_oscar_test_vocab32k.txt.bpe \
    --destdir ./vocab32k \
    --workers 8
```

Vocab50k process data

```
fairseq-preprocess \
    --only-source \
    --srcdict ./vocab/books_wikipedia_v50k_sen10M.spm.bpe_fair.vocab \
    --trainpref corpus_wiki_books_oscar_train_vocab50k.txt.bpe \
    --validpref corpus_wiki_books_oscar_test_vocab50k.txt.bpe \
    --destdir ./vocab50k \
    --workers 8
```

## Train the model - small wikipedia





### AWS configuration for small wikipedia model

#### p3 instance - linear decay


```
TOTAL_UPDATES=125000    # Total number of training steps
WARMUP_UPDATES=10000    # Warmup the learning rate over this many updates
PEAK_LR=0.0005          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=16          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/datasets/roberta_data/wiki_model/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model/logs/

fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test \
    --save-dir $SAVE_DIR --tensorboard-logdir $LOGS_DIR --keep-last-epochs 10

```

#### g4 instance - wiki triangular


```
TOTAL_UPDATES=125000    # Total number of training steps
STEP_SIZE=5000
BASE_LR=0.0001
PEAK_LR=0.001          # Peak learning rate, adjust as needed
LR_SHRINK=0.6           # better is 0.9
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=16          # Increase the batch size 16x


DATA_DIR=/mnt/efs/fs1/bert_model/datasets/roberta_data/wiki_model/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model_tri
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model_tri/logs/

fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler triangular --lr $BASE_LR --max-lr $PEAK_LR \
    --lr-period-updates $STEP_SIZE --lr-shrink $LR_SHRINK --shrink-min \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR \
    --ddp-backend=no_c10d
```

#### g4 instance - wiki cosine



```
TOTAL_UPDATES=30000    # Total number of training steps
WARMUP_UPDATES=1000
BASE_LR=0.00005
PEAK_LR=0.0005          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=16          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/datasets/roberta_data/wiki_model/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model_cos
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model_cos/logs/

fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler cosine --warmup-updates $WARMUP_UPDATES --lr-period-updates 1000 --t-mult 2 --lr-shrink 0.9 --lr $BASE_LR --max-lr $PEAK_LR \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR\
    --ddp-backend=no_c10d
```

### Odra settings

```
TOTAL_UPDATES=30000    # Total number of training steps
WARMUP_UPDATES=10000    # Warmup the learning rate over this many updates
PEAK_LR=0.0005          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=6        
UPDATE_FREQ=16          

DATA_DIR=/mnt/hdd/goodwrite_nas/bert_model/datasets/roberta_data/wiki_model

fairseq-train $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test --tensorboard-logdir ./logs 


TOTAL_UPDATES=50000    # Total number of training steps
WARMUP_UPDATES=5000
PEAK_LR=0.001          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=6        
UPDATE_FREQ=16          

DATA_DIR=/mnt/hdd/goodwrite_nas/bert_model/datasets/roberta_data/wiki_model


#after 32184 steps, 12 epoch
fairseq-train $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --reset-lr-scheduler \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test --tensorboard-logdir ./logs 



fairseq-train $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --reset-lr-scheduler \
    --lr-scheduler triangular --lr $BASE_LR --max-lr $MAX_LR \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test --tensorboard-logdir ./logs 

## Train wiki books oscar model 

Tech: 

* Pytorch 1.5
* Apex 
* CUDA 10.2
* fairseq 0.9


### Experiment 1 - Linear decay


Vocab: upper 32k tokens
Train on AWS p3.16xlarge

Efective batch size = MAX_SENTENCES*UPDATE_FREQ* num_gpu = 16\*64\*8 = 8192

First start experiments with linear scheduler, run for 50k updates


```
TOTAL_UPDATES=50000    # Total number of training steps
WARMUP_UPDATES=10000    # Warmup the learning rate over this many updates
PEAK_LR=0.0005          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=64          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/roberta_data/wiki_books_oscar/vocab32k/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_linear/
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_linear/logs/

fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test \
    --save-dir $SAVE_DIR --tensorboard-logdir $LOGS_DIR --keep-last-epochs 10 \
    --ddp-backend=no_c10d \


```




```
tensorboard dev upload --logdir $LOGS_DIR \
    --name "Herber polish Roberta model #1 - wiki, books, oscar 32k vocab32k" \
    --description "Wiki, books, oscar 32k, vocab 32k, upper, linear schedule "

```




### Experiment 2 - cyclic triangular

Vocab: upper 32k tokens

Train on AWS p3.16xlarge

Cyclic triangular schedule, 5000 steps for rise to peek lr and fall to base lr, after each 5k stpe shirnk peak and base lr

Efective batch size = MAX_SENTENCES*UPDATE_FREQ* num_gpu = 16\*64\*8 = 8192



```
TOTAL_UPDATES=50000    # Total number of training steps
STEP_SIZE=5000
BASE_LR=0.0001
PEAK_LR=0.001          # Peak learning rate, adjust as needed
LR_SHRINK=0.8           # 
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=64          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/roberta_data/wiki_books_oscar/vocab32k/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_tri/
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_tri/logs/


fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler triangular --lr $BASE_LR --max-lr $PEAK_LR \
    --lr-period-updates $STEP_SIZE --lr-shrink $LR_SHRINK --shrink-min \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR \
    --ddp-backend=no_c10d
```

```
tensorboard dev upload --logdir $LOGS_DIR \
    --name "Herber polish Roberta model #2 - wiki, books, oscar 32k vocab32k" \
    --description "Wiki, books, oscar 32k, vocab 32k, upper, tri schedule "






### Experiment 3 - cyclic cosine

Vocab: upper 32k tokens

Train on AWS p3.16xlarge

Cyclic cosine schedule, 5000 steps for rise to peek lr and fall to base lr, after each 5k stpe shirnk peak and base lr

Efective batch size = MAX_SENTENCES*UPDATE_FREQ* num_gpu = 16\*64\*8 = 8192

```
TOTAL_UPDATES=50000    # Total number of training steps
STEP_SIZE=2500
WARMUP_UPDATES=2500 # same as triangular
BASE_LR=0.0001
PEAK_LR=0.001          # Peak learning rate, adjust as needed
LR_SHRINK=0.8           # 
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=64          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/roberta_data/wiki_books_oscar/vocab32k/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_cos1/
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_cos1/logs/


fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler cosine --lr $BASE_LR --max-lr $PEAK_LR \
    --warmup-updates $WARMUP_UPDATES  \
    --lr-period-updates $STEP_SIZE --t-mult 1 --lr-shrink $LR_SHRINK  \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR \
    --ddp-backend=no_c10d
```

```
tensorboard dev upload --logdir $LOGS_DIR \
    --name "Herber polish Roberta model #3 - wiki, books, oscar 32k vocab32k" \
    --description "Wiki, books, oscar 32k, vocab 32k, upper, cosine schedule step=2500"

```



### Experiment 4 - cyclic cosine

Vocab: upper 32k tokens

Train on AWS p3.16xlarge

Cyclic cosine schedule, 5000 steps for rise to peek lr and fall to base lr, after each 5k stpe shirnk peak and base lr
Half the peak lr, because previous experiment not convergence


Add num-workers to read data from disk


Efective batch size = MAX_SENTENCES*UPDATE_FREQ* num_gpu = 16\*64\*8 = 8192

```
TOTAL_UPDATES=50000    # Total number of training steps
STEP_SIZE=2500
WARMUP_UPDATES=2500 # same as triangular
BASE_LR=0.0001
PEAK_LR=0.0005          # Peak learning rate, adjust as needed
LR_SHRINK=0.8           # 
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=64          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/roberta_data/wiki_books_oscar/vocab32k/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_cos1_2/
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_cos1_2/logs/


fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler cosine --lr $BASE_LR --max-lr $PEAK_LR \
    --warmup-updates $WARMUP_UPDATES  \
    --lr-period-updates $STEP_SIZE --t-mult 1  --lr-shrink $LR_SHRINK  \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR \
    --ddp-backend=no_c10d --num-workers 2
```

```
tensorboard dev upload --logdir $LOGS_DIR \
    --name "Herber polish Roberta model #4 - wiki, books, oscar 32k vocab32k" \
    --description "Wiki, books, oscar 32k, vocab 32k, upper, cosine schedule step=2500 half lr=0.0005"

```



### Experiments 4, 5, 6 - cyclic cosine




Vocab: upper 32k tokens

Train on AWS p3.16xlarge

Cyclic cosine schedule, 1000 steps for rise to peek lr and fall to base lr, 


Add num-workers to read data from disk


Efective batch size = MAX_SENTENCES*UPDATE_FREQ* num_gpu = 16\*64\*8 = 8192

```
TOTAL_UPDATES=50000    # Total number of training steps
STEP_SIZE=1000
WARMUP_UPDATES=1000 
BASE_LR=0.0001
PEAK_LR=0.001          # Peak learning rate, adjust as needed
LR_SHRINK=0.8           # 
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=64          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/roberta_data/wiki_books_oscar/vocab32k/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_cos1_4/
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_cos1_4/logs/


fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.9 \
    --lr-scheduler cosine --lr $BASE_LR --max-lr $PEAK_LR \
    --warmup-updates $WARMUP_UPDATES  \
    --lr-period-updates $STEP_SIZE --t-mult 2  --lr-shrink $LR_SHRINK  \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR \
    --ddp-backend=no_c10d --num-workers 2
```

```
tensorboard dev upload --logdir $LOGS_DIR \
    --name "Herber polish Roberta model #6 - wiki_books_oscar_32k_cos1_4"  \
    --description "Wiki, books, oscar 32k, vocab 32k, upper, cosine schedule step=1000 t-mult=2 clip-norm=0.9"

```

### Experiment 7 - Final tri vocab 32k - cyclic triangular

Vocab: upper 32k tokens

Train on AWS p3.16xlarge

Efective batch size = MAX_SENTENCES*UPDATE_FREQ* num_gpu = 16\*64\*8 = 8192


```
TOTAL_UPDATES=125000    # Total number of training steps
STEP_SIZE=5000
BASE_LR=0.0001
PEAK_LR=0.001          # Peak learning rate, adjust as needed
LR_SHRINK=0.8           # 
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=64          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/roberta_data/wiki_books_oscar/vocab32k/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_tri_full/
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_tri_full/logs/


fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler triangular --lr $BASE_LR --max-lr $PEAK_LR \
    --lr-period-updates $STEP_SIZE --lr-shrink $LR_SHRINK --shrink-min \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR \
    --ddp-backend=no_c10d
```

```
tensorboard dev upload --logdir $LOGS_DIR \
    --name "Herber polish Roberta model #7 final tri - wiki, books, oscar 32k vocab32k" \
    --description "Wiki, books, oscar 32k, vocab 32k, upper, tri schedule "






### Experiments 8 - cyclic cosine full




Vocab: upper 32k tokens

Train on AWS p3.16xlarge
```
TOTAL_UPDATES=125000    # Total number of training steps
STEP_SIZE=1000
WARMUP_UPDATES=5000 
BASE_LR=0.00001
PEAK_LR=0.0007          # Peak learning rate, adjust as needed
LR_SHRINK=0.7           # 
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=64          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/roberta_data/wiki_books_oscar/vocab32k/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_cos1_5/
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_cos1_5/logs/


fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.9 \
    --lr-scheduler cosine --lr $BASE_LR --max-lr $PEAK_LR \
    --warmup-updates $WARMUP_UPDATES  \
    --lr-period-updates $STEP_SIZE --t-mult 2  --lr-shrink $LR_SHRINK  \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR \
    --ddp-backend=no_c10d --num-workers 2
```

```
tensorboard dev upload --logdir $LOGS_DIR \
    --name "Herber polish Roberta model #8 full 125k - wiki_books_oscar_32k_cos1_5"  \
    --description "Wiki, books, oscar 32k, vocab 32k, upper, cosine schedule  t-mult=2 clip-norm=0.9"

```

### Experiment 9 - Linear full


Vocab: upper 32k tokens
Train on AWS p3.16xlarge

Efective batch size = MAX_SENTENCES*UPDATE_FREQ* num_gpu = 16\*64\*8 = 8192

First start experiments with linear scheduler, run for 50k updates


```
TOTAL_UPDATES=125000    # Total number of training steps
WARMUP_UPDATES=10000    # Warmup the learning rate over this many updates
PEAK_LR=0.001          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=64          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/roberta_data/wiki_books_oscar/vocab32k/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_linear_full/
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_32k_linear_full/logs/

fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.9 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test \
    --save-dir $SAVE_DIR --tensorboard-logdir $LOGS_DIR  \
    --ddp-backend=no_c10d \

```

```
tensorboard dev upload --logdir $LOGS_DIR \
    --name "Herber polish Roberta model #9 full 125k - wiki_books_oscar_32k_linear_full"  \
    --description "Wiki, books, oscar, vocab 32k, upper, linear clip-norm=0.9"

```


### Experiment 10 - Linear 50k


Vocab: upper 50k tokens
Train on AWS p3.16xlarge

Efective batch size = MAX_SENTENCES*UPDATE_FREQ* num_gpu = 16\*64\*8 = 8192

First start experiments with linear scheduler, run for 50k updates


```
TOTAL_UPDATES=50000    # Total number of training steps
WARMUP_UPDATES=10000    # Warmup the learning rate over this many updates
PEAK_LR=0.001          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=64          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/roberta_data/wiki_books_oscar/vocab50k/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_50k_linear50k/
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_books_oscar_50k_linear50k/logs/

fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.9 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test \
    --save-dir $SAVE_DIR --tensorboard-logdir $LOGS_DIR  \
    --ddp-backend=no_c10d \

```

```
tensorboard dev upload --logdir $LOGS_DIR \
    --name "Herber polish Roberta model #10 - wiki_books_oscar_50k_linear50K"  \
    --description "Wiki, books, oscar, vocab 50k, 50k steps, upper, linear clip-norm=0.9"

```


Experiments (https://docs.google.com/spreadsheets/d/1fBhELqDB1kAxLCBvzeVM4OhqO4zx-meRUVljK1YZfF8/edit#gid=0)

* experiment 1 - linear schedule peek_lr=5e-4  updates=50000, bsz=8192, test convergence speed of linear schedule, try to find optimal speed https://tensorboard.dev/experiment/pNCxXW9zQKKEoxkNN1LeKg/#scalars&tagFilter=lr%7C.*loss%24%7Cppl
* experiment 2 - cyclic triangular schedule updates=50000, bsz=8192, cyclic step=5000 test convergence speed of linear schedule, try to find optimal speed https://tensorboard.dev/experiment/pNCxXW9zQKKEoxkNN1LeKg/#scalars&tagFilter=lr%7C.*loss%24%7Cppl
* experiment 3 (wiki_books_oscar_32k_cos1) - cyclic cosine schedule,  updates=50000, bsz=8192, cyclic step=2500- test convergence speed of linear schedule, try to find optimal speed, should be similar to tirangular schedule in 5000 steps goes up and down with lr https://tensorboard.dev/experiment/SY64gY46SKq7wGohxgjlgg/#scalars&tagFilter=lr%7C.*loss%24%7Cppl after 23k steps experiment was stopped, loss jumped and plateau 
* experiment 4(wiki_books_oscar_32k_cos1_2) - similar to experiment 3 but peak learning rata was changed from 0.001 to 0.0005, I hope this will prevent loss jump and plateau  https://tensorboard.dev/experiment/wpK8EgmqQKiqJABKAndDwQ/
* experiment 5 (wiki_books_oscar_32k_cos1_3) - similar to experiment 3, but decrese the step size and set t-mult to 2, we want restarts more frequenty https://tensorboard.dev/experiment/NzyYnXBQQ7qbBFYdu57byg/#scalars
* experiment 6 (wiki_books_oscar_32k_cos1_4) - similar to experiment 5, but with clip-norm 0.9